In [3]:
import torch
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
torch.cuda.set_device(0)

In [10]:
from ultralytics import YOLO
from ray import tune

current_dir = os.getcwd()
relative_path = 'Hard-Hat-Workers-8\custom_hard_hat.yaml'
dataset_path = os.path.abspath(os.path.join(current_dir, 
                                            os.pardir, 
                                            relative_path))

# Load the model
model = YOLO("yolov8n.pt")

# Hyperparameter Tuning with Ray Tune:
result = model.tune(
    data=dataset_path,
    gpu_per_trial=1,
    max_samples=5,
    space={
      "lr0": tune.uniform(1e-5, 1e-1),
      "weight_decay": tune.uniform(0.0, 0.001),
      "momentum": tune.uniform(0.6, 0.7),
      "degrees": tune.uniform(0.0, 30.0)
      },
    train_args={
      "epochs": 10, 
      "imgsz":640, 
      "batch":32
      }
)

# https://docs.ultralytics.com/tasks/detect/#val
# https://docs.ultralytics.com/modes/predict/


2023-05-13 21:45:37,364	WARNING worker.py:1986 -- Warning: The actor ImplicitFunc is very large (12 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.
2023-05-13 21:45:37,405	WARNING util.py:244 -- The `start_trial` operation took 0.640 s, which may be a performance bottleneck.
(_tune pid=23060) Ultralytics YOLOv8.0.99  Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 2060, 6144MiB)
(_tune pid=23060) yolo\engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=e:\files_main\projects_ml\hard-hat-detection\Hard-Hat-Workers-8\custom_hard_hat.yaml, epochs=10, patience=50, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, overlap

Trial name,date,done,epoch,hostname,iterations_since_restore,metrics/mAP50(B),metrics/mAP50-95(B),metrics/precision(B),metrics/recall(B),node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,training_iteration,trial_id,val/box_loss,val/cls_loss,val/dfl_loss
_tune_6f88c_00000,2023-05-13_22-05-13,True,9,DaveJammin_TUF,11,0.970467,0.645593,0.957175,0.904166,127.0.0.1,23060,1165.09,27.0841,1165.09,1683986713,11,6f88c_00000,,,
_tune_6f88c_00001,2023-05-13_22-20-00,False,5,DaveJammin_TUF,6,0.9621,0.58061,0.94932,0.89854,127.0.0.1,23060,884.32,95.1157,884.32,1683987600,6,6f88c_00001,1.39005,0.667,1.21489


(_tune pid=23060) 
(_tune pid=23060)       Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
  0%|          | 0/154 [00:00<?, ?it/s]
       2/10       4.3G      1.292      1.031      1.207        123        640:   1%|          | 1/154 [00:00<01:07,  2.27it/s]
       2/10       4.3G      1.339     0.9857      1.189        179        640:   1%|▏         | 2/154 [00:00<01:02,  2.41it/s]
       2/10       4.3G      1.332     0.9708      1.189        166        640:   2%|▏         | 3/154 [00:01<01:03,  2.37it/s]
       2/10       4.3G      1.336     0.9578      1.187        164        640:   3%|▎         | 4/154 [00:01<01:03,  2.37it/s]
       2/10       4.3G      1.334      0.959      1.182        131        640:   3%|▎         | 5/154 [00:02<01:02,  2.37it/s]
       2/10       4.3G      1.335     0.9502      1.182        186        640:   4%|▍         | 6/154 [00:02<01:01,  2.39it/s]
       2/10       4.3G      1.335     0.9467      1.191        193        640:   5%

In [15]:
result.get_best_result(metric='metrics/mAP50(B)', mode='max')

Result(
  metrics={'metrics/precision(B)': 0.9557885177710965, 'metrics/recall(B)': 0.9399437771530795, 'metrics/mAP50(B)': 0.9815966113849529, 'metrics/mAP50-95(B)': 0.6830291017293038, 'epoch': 9, 'done': True, 'trial_id': '6f88c_00002', 'experiment_tag': '2_degrees=3.1204,lr0=0.0078,momentum=0.6143,weight_decay=0.0000'},
  path='e:\\files_main\\projects_ml\\hard-hat-detection\\src\\runs\\_tune_2023-05-13_21-45-34\\_tune_6f88c_00002_2_degrees=3.1204,lr0=0.0078,momentum=0.6143,weight_decay=0.0000_2023-05-13_22-26-44',
  checkpoint=None
)

In [ ]:
# Load the model.
model = YOLO('yolov8n.pt')
 
# Training.
results = model.train(
   data='../Hard-Hat-Workers-8/custom_hard_hat.yaml',
   imgsz=640,
   epochs=10,
   batch=8,
   name='yolov8n_custom'
   )